In [7]:
import requests

In [ ]:
'''
1. Launch the website: https://www.solrenview.com/SolrenView/mainFr.php?siteId=6073
2. Trigger the event by clicking "Export Data"
3. Developer tools -> Network -> Fetch/XHR
'''
request_URL = 'https://www.solrenview.com/srvp/FSC/SRV/getAnalyticsInfo.php?siteId=6073&queryNo=4&startIntervalTs=2025-10-22%2004:00:00&endIntervalTs=2025-10-22%2005:00:00&endDuration=2025-10-23%2000:00:00&dataInterval=3600&deviceindices=0^35346^Inverter%201%20%20[1013821839182%20PVI%2060TL]^3;0^35347^Inverter%202%20%20[1013821839191%20PVI%2060TL]^3&parList=ecount&deviceType=0^35346^Inverter%201%20[1013821839182%20PVI%2060TL]^3;0^35347^Inverter%202%20[1013821839191%20PVI%2060TL]^3&hasRg=0&maxLimit=360000&gauge=0+&dataIntervalStr=Minute'

In [3]:
url = 'https://www.solrenview.com/srvp/FSC/SRV/getAnalyticsInfo.php'

In [4]:
params = {
 'siteId': '6073',
 'queryNo': '4',
 'startIntervalTs': '2025-10-22 04:00:00',
 'endIntervalTs': '2025-10-22 04:05:00',
 'endDuration': '2025-10-23 00:00:00',
 'dataInterval': '300',
 'deviceindices': '0^35346^Inverter 1  [1013821839182 PVI 60TL]^3;0^35347^Inverter 2  [1013821839191 PVI 60TL]^3',
 'parList': 'vdc;vdc2;vdc3;vdc4;idc;idc2;idc3;idc4',
 'deviceType': '0^35346^Inverter 1 [1013821839182 PVI 60TL]^3;0^35347^Inverter 2 [1013821839191 PVI 60TL]^3',
 'hasRg': '0',
 'maxLimit': '360000',
 'gauge': '0 ',
 'dataIntervalStr': 'Minute'
}

In [ ]:
'''
Developer tools -> Network -> Fetch/XHR -> Headers -> Request Headers
'''
headers = {
 'Cookie': 'siteId=6073; PHPSESSID=0ooqna8h758bs6c4jl9gl6reu7; _ga=GA1.2.792316693.1761186174; _gid=GA1.2.2038752395.1761186174; _gat=1; _ga_L4S8LG26JM=GS2.2.s1761186174$o1$g1$t1761186879$j60$l0$h0',
 'Referer': 'https://www.solrenview.com/srvp/FSC/SRV/analyticsT.php?siteId=6073&CurrentTab=3',
 'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36'
}

In [8]:
response = requests.get(url, params=params, headers=headers)

In [9]:
response.ok

True

In [11]:
response.json().keys()

dict_keys(['chart', 'categories', 'dataset', 'styles'])

In [12]:
response.json()['dataset']

[{'seriesname': ' Inverter 1  [1013821839182 PVI 60TL] - DC Voltage1(Volts)',
  'renderas': 'Line',
  'parentyaxis': 'P',
  'data': [{'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': ''},
   {'value': '352.667'},
   {'value': '486